<a href="https://colab.research.google.com/github/nntadotzip/vi-augmentation/blob/main/BertPairClassification_SynonymReplacementMethod_5703sem0of1to1999and5000to7162__8627sem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Install

**Models**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install datasets==1.0.1
!pip install transformers==3.1.0

In [3]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric

os.environ["TOKENIZERS_PARALLELISM"] = "false"

PyTorch version 1.10.0+cu111 available.
TensorFlow version 2.8.0 available.


In [4]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.0 GB  | Proc size: 1.6 GB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total 15109MB


**Cosine Similarity**

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [6]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Load the Dataset

In [7]:
augmented_sem0_1to1999_1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/backtranslation_results/backtrans_1to1999_sem0.csv', encoding='utf8')
augmented_sem0_1to1999_2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/backtranslation_results/backtrans_1to1999_sem0_SWAPPED.csv', encoding='utf8')
augmented_sem0_5000to7162_1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/backtranslation_results/backtrans_5000to7162_sem0.csv', encoding='utf8')
augmented_sem0_5000to7162_2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/backtranslation_results/backtrans_5000to7162_sem0_SWAPPED.csv', encoding='utf8')

In [8]:
augmented_sem0_1to1999_1 = augmented_sem0_1to1999_1[['q1_en_to_vi', 'q2_fr_to_vi']]
augmented_sem0_1to1999_1['ans_question_1'] = ''
augmented_sem0_1to1999_1['ans_question_2'] = ''
augmented_sem0_1to1999_1['cosine_similarity'] = ''
augmented_sem0_1to1999_1['manual_label'] = 0
augmented_sem0_1to1999_1 = augmented_sem0_1to1999_1[['q1_en_to_vi', 'q2_fr_to_vi', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_sem0_1to1999_1 = augmented_sem0_1to1999_1.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)

augmented_sem0_1to1999_2 = augmented_sem0_1to1999_2[['q1_en_to_vi', 'q2_fr_to_vi']]
augmented_sem0_1to1999_2['ans_question_1'] = ''
augmented_sem0_1to1999_2['ans_question_2'] = ''
augmented_sem0_1to1999_2['cosine_similarity'] = ''
augmented_sem0_1to1999_2['manual_label'] = 0
augmented_sem0_1to1999_2 = augmented_sem0_1to1999_2[['q1_en_to_vi', 'q2_fr_to_vi', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_sem0_1to1999_2 = augmented_sem0_1to1999_2.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)

augmented_sem0_5000to7162_1 = augmented_sem0_5000to7162_1[['q1_en_to_vi', 'q2_fr_to_vi']]
augmented_sem0_5000to7162_1['ans_question_1'] = ''
augmented_sem0_5000to7162_1['ans_question_2'] = ''
augmented_sem0_5000to7162_1['cosine_similarity'] = ''
augmented_sem0_5000to7162_1['manual_label'] = 0
augmented_sem0_5000to7162_1 = augmented_sem0_5000to7162_1[['q1_en_to_vi', 'q2_fr_to_vi', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_sem0_5000to7162_1 = augmented_sem0_5000to7162_1.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)

augmented_sem0_5000to7162_2 = augmented_sem0_5000to7162_2[['q1_en_to_vi', 'q2_fr_to_vi']]
augmented_sem0_5000to7162_2['ans_question_1'] = ''
augmented_sem0_5000to7162_2['ans_question_2'] = ''
augmented_sem0_5000to7162_2['cosine_similarity'] = ''
augmented_sem0_5000to7162_2['manual_label'] = 0
augmented_sem0_5000to7162_2 = augmented_sem0_5000to7162_2[['q1_en_to_vi', 'q2_fr_to_vi', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_sem0_5000to7162_2 = augmented_sem0_5000to7162_2.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [9]:
augmented_sem0 = augmented_sem0_1to1999_1.append(augmented_sem0_1to1999_2, sort=False, ignore_index=True)
augmented_sem0 = augmented_sem0.append(augmented_sem0_5000to7162_1, sort=False, ignore_index=True)
augmented_sem0 = augmented_sem0.append(augmented_sem0_5000to7162_2, sort=False, ignore_index=True)
augmented_sem0

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Các biện pháp cơ bản để mang lại đồng bằng sông Hồng sẽ sớm trở thành khu vực sản xuất thực phẩm, bất động sản là: chú ý đến các sản phẩm và thị trường.|.Thay đổi cấu trúc của cây trồng và cấu trúc theo mùa.|.Hãy chú ý đến môi trường và bảo vệ tài nguyên đất.|.Mạnh mẽ phát triển văn hóa trồng trọt.",,,,0
1,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Hướng về định hướng tái cấu trúc trong lĩnh vực kinh tế nội bộ ở Delta Red River là: |Phát triển và hiện đại hóa nông nghiệp, gắn phát triển với ngành công nghiệp chế biến.|.Phát triển và kịp thời của các nhà khai thác công nghiệp chuyển đổi, trong khi các ngành công nghiệp và dịch vụ khác có liên quan đến điều kiện nông nghiệp của hàng hóa.|.Phát triển và hiện đại hóa khai thác công nghiệp, gắn nó với các sản phẩm nông nghiệp.|.Phát triển và hiện đại hóa ngành chế biến và khai thác.",,,,0
2,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Câu hỏi kinh tế xã hội là mối quan tâm lớn ở đồng bằng sông Hồng trong giai đoạn hiện tại: |Các lĩnh vực chính của sản xuất thực phẩm và sản xuất thực phẩm.|.Dân số Đông, diện tích đất canh tác hạn chế.|.Trình độ chuyên môn với cường độ cao.|.Nơi tập trung vào nhiều trung tâm kinh tế, văn hóa và chính trị lớn trên toàn quốc.",,,,0
3,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Dựa trên Địa lý Atlat Việt Nam, vui lòng cho biết các tỉnh nào sau đây của đồng bằng sông Hồng không liền kề với biển?|.Hưng Yên, Hải Phòng.|.Hà Nam, Bắc Ninh.|.Hà Nam, Ninh Bình.|.Đinh Man, Bắc Ninh.",,,,0
4,"Dân số và lao động |Dân số Đông, nguồn lao động dồi dào và có trình độ.|.Lực lượng lao động lớn nhất trong cả nước.|.Công nhân có trình độ cao nhất trên toàn quốc.|.Lao động tập trung chủ yếu các thành phố.","Dựa trên Atlat địa lý của Việt Nam, vui lòng cho biết trung tâm công nghiệp nào trong đồng bằng sông Hồng có giá trị sản xuất công nghiệp từ 40 đến 120 nghìn tỷ đồng không?|.Hà Nội.|.Hải Phòng.|.Phúc Yên.|.Ninh Bac.|.Ninh Bac.|.",,,,0
...,...,...,...,...,...,...
5701,"Kim ngạch xuất nhập khẩu của đất nước chúng tôi liên tục tăng chủ yếu do thị trường thế giới ngày càng mở rộng.|.Đa dạng hóa các đối tượng tham gia vào các hoạt động xuất nhập khẩu.|.Tăng cường nhập khẩu các dòng máy móc, toàn bộ thiết bị và hàng tiêu dùng.|.Sự phát triển của nền kinh tế trong nước và đổi mới trong các cơ chế quản lý.",Các yếu tố khí hậu cũng ảnh hưởng đến các tổ chức lãnh thổ công nghiệp vì |Chi tiêu lựa chọn kỹ thuật và công nghệ.|.Ảnh hưởng đến nguyên liệu thô.|.Thảm họa thường gây thiệt hại cho sản xuất công nghiệp.|.Thống trị quy mô và cơ cấu của các doanh nghiệp công nghiệp.,,,,0
5702,Các trung tâm công nghiệp ở vùng trung du phía bắc và miền núi phía bắc được phát triển chủ yếu trên cơ sở công nhân có kinh nghiệm trong sản xuất.|.Vị trí chiến lược liền kề miền Nam Trung Quốc |Giàu tài nguyên và địa điểm địa lý thuận lợi của Khoáng sản.|.Cơ sở hạ tầng tương đối hoàn thành.,Yếu tố quan trọng nhất dẫn đến sự khác biệt về phân phối trà và cao su ở nước ta: |thời tiết.|.Đất.|.Trái đất.|.Nước uống.|.,,,,0
5703,"Kim ngạch xuất nhập khẩu của đất nước chúng tôi liên tục tăng chủ yếu do thị trường thế giới ngày càng mở rộng.|.Đa dạng hóa các đối tượng tham gia vào các hoạt động xuất nhập khẩu.|.Tăng cường nhập kh

In [10]:
original_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/prep/vietjackGeo/labeled_1to1999/labeled_1to2000_csvformat.csv', encoding='utf8')
original_df.head(2)

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Biện pháp cơ bản để đưa đồng bằng sông Hồng sớm trở thành vùng sản xuất lương thực, thực phầm hàng hóa là:|quan tâm đến chất lương sản phẩm và thị trường.|thay đổi cơ cấu cây cây trồng và cơ cấu mùa vụ.|chú ý đến môi trường và bảo vệ tài nguyên đất.|phát triển mạnh cây vụ đông.","dân số đông, nguồn lao động dồi dào và có trình độ.",thay đổi cơ cấu cây cây trồng và cơ cấu mùa vụ.,"0,71565396",0.0
1,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Tại sao việc làm là một trong những vấn đề nan giải ở Đồng bằng sông Hồng nhất là ở khu vực thành thị?|Do dân nhập cư đông.|Do dân số đông, kết cấu dân số trẻ.|Do nền kinh tế còn chậm phát triển.|Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","0,8153097",1.0


In [11]:
augmented_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/Augmentation/prep/vietjackGeo/final_augment/syn_augmented_1to1999.csv', encoding='utf8')
augmented_df

,sentence1,sentence2,ans_question_1,ans_question_2,manual_label,augmented1,augmented2
0,"“dân cư và lao động”|dân số đông, nguồn lao động dồi dào và có trình độ.|nguồn lao động lớn nhất cả nước.|lao động có trình độ cao nhất cả nước.|lao động tập trung chủ yếu ở các thành phố lớn.","Tại sao việc làm là một trong những vấn đề nan giải ở Đồng bằng sông Hồng nhất là ở khu vực thành thị?|Do dân nhập cư đông.|Do dân số đông, kết cấu dân số trẻ.|Do nền kinh tế còn chậm phát triển.|Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm",1.0,"“ dân cư và lao động ” | dân cư đông , nguồn lao động dồi dào và có trình độ .| nguồn lao động lớn nhất cả nước .| lao động có trình độ cao nhất cả nước .| lao động tập trung chủ yếu ở các thành phố lớn .","tại sao việc làm là một trong những vấn đề nan giải ở đồng bằng sông hồng nhất là ở khu vực thành thị ?| do dân nhập cư đông .| do dân cư đông , kết cấu dân cư trẻ .| do nền kinh tế còn chậm phát triển .| do dân cư đông , kết cấu dân cư trẻ trong điều kiện kinh tế chậm"
1,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió mạnh tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .
2,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng ngoại ô nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .
3,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| cao nguyên .
4,"Bão, lũ lụt, hạn hán, gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng|Đồng bằng sông Hồng.|Tây Bắc.|Duyên hải miền Trung.|Tây Nguyên",Vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió Tây khô nóng?|Bắc Trung Bộ.|Đông Bắc.|Đông Nam Bộ.|Tây Nguyên.,Duyên hải miền Trung.,Bắc Trung Bộ.,1.0,"bão , lũ lụt , hạn hán , gió tây khô nóng là thiên tai xảy ra chủ yếu ở vùng ngoại ô | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió mạnh tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .
...,...,...,...,...,...,...,...
8622,"Việc phát triển kinh tế - xã hội vùng dân tộc ít người ở nước ta cần được chú trọng hơn nữa do|các dân tộc ít người đóng vai trò rất quan trọng trong việc đảm bảo an ninh quốc phòng.|một số dân tộc ít người có những kinh nghiệm sản xuất quí báu.|sự phát triển kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể, mức sống của bộ phận dân tộc ít người thấp.|trước đây chúng ta chưa chú trọng vấn đề này.","Đặc điểm nào không đúng với dân cư, dân tộc ở nước t

In [12]:
augmented_df =  augmented_df[['augmented1', 'augmented2', 'ans_question_1', 'ans_question_2', 'manual_label']]
augmented_df['cosine_similarity'] = ''
augmented_df  = augmented_df[['augmented1', 'augmented2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label']]
augmented_df = augmented_df.set_axis(['question_1', 'question_2', 'ans_question_1', 'ans_question_2', 'cosine_similarity','manual_label'], axis=1, inplace=False)
augmented_df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"“ dân cư và lao động ” | dân cư đông , nguồn lao động dồi dào và có trình độ .| nguồn lao động lớn nhất cả nước .| lao động có trình độ cao nhất cả nước .| lao động tập trung chủ yếu ở các thành phố lớn .","tại sao việc làm là một trong những vấn đề nan giải ở đồng bằng sông hồng nhất là ở khu vực thành thị ?| do dân nhập cư đông .| do dân cư đông , kết cấu dân cư trẻ .| do nền kinh tế còn chậm phát triển .| do dân cư đông , kết cấu dân cư trẻ trong điều kiện kinh tế chậm","dân số đông, nguồn lao động dồi dào và có trình độ.","Do dân số đông, kết cấu dân số trẻ trong điều kiện kinh tế chậm",,1.0
1,"bão , lũ lụt , hạn hán , gió mạnh tây khô nóng là thiên tai xảy ra chủ yếu ở vùng | đồng bằng sông hồng .| tây bắc .| duyên hải miền trung .| tây nguyên",vùng nào ở nước ta chịu ảnh hưởng mạnh mẽ nhất của gió mạnh tây khô nóng ?| bắc trung bộ .| đông bắc .| đông nam bộ .| tây nguyên .,Duyên hải miền Trung.,Bắc Trung Bộ.,,1.0


In [13]:
merged_df = original_df
merged_df = merged_df.append(augmented_sem0, sort=False, ignore_index=True)
merged_df = merged_df.append(augmented_df, sort=False, ignore_index=True)

In [14]:
merged_df.tail(2)

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
16330,"việc phát ra kinh tế - xã hội vùng dân tộc ít người ở nước ta cần được chú trọng hơn nữa do | các dân tộc ít người đóng vai trò rất quan trọng trong việc đảm bảo an ninh quốc phòng .| một số dân tộc ít người có những kinh nghiệm sản xuất quí báu .| sự phát ra kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể , mức sống của bộ phận dân tộc ít người thấp .| trước đây chúng ta chưa chú trọng vấn đề này .","đặc điểm nào không đúng với dân cư , dân tộc ở nước ta ?| các dân tộc luôn phát huy truyền thống sản xuất .| các dân tộc luôn đoàn kết bên nhau .| chất lượng đời sống của các dân tộc ít người đã ở mức cao .| sự phát triển kinh tế - xã hội giữa các vùng còn chênh lệch .","sự phát triển kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể, mức sống của bộ phận dân tộc ít người thấp.",Chất lượng đời sống của các dân tộc ít người đã ở mức cao.,,1.0
16331,"việc phát ra kinh tế - xã hội vùng dân tộc ít người ở nước ta cần được chú trọng hơn nữa do | các dân tộc ít người đóng vai trò rất quan trọng trong việc đảm bảo an ninh quốc phòng .| một số dân tộc ít người có những kinh nghiệm sản xuất quí báu .| sự phát ra kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể , mức sống của bộ phận dân tộc ít người thấp .| trước đây chúng ta chưa chú trọng vấn đề này .","đặc điểm nào không đúng với dân cư , dân tộc ở nước ta ?| các dân tộc luôn phát huy truyền thống sản xuất .| các dân tộc luôn đoàn kết bên nhau .| chất lượng đời sống của các dân tộc ít người đã ở mức cao .| sự phát ra kinh tế - xã hội giữa các vùng còn chênh lệch .","sự phát triển kinh tế - xã hội giữa các dân tộc hiện có sự chênh lệch đáng kể, mức sống của bộ phận dân tộc ít người thấp.",Chất lượng đời sống của các dân tộc ít người đã ở mức cao.,,1.0


In [15]:
merged_df = merged_df.sample(frac = 1)
merged_df = merged_df.reset_index(drop=True)
merged_df

,question_1,question_2,ans_question_1,ans_question_2,cosine_similarity,manual_label
0,"Đâu là đặc điểm thực sự của các hoạt động bão tại Việt Nam?|.Mùa bão bắt đầu từ tháng 1 và kết thúc vào tháng 11.|.Mùa bão chậm chậm từ phía nam đến phía bắc .70% những cơn bão trong mùa được tập trung vào những tháng VIII, IX, X. |Trung bình, 10 đến 12 cơn bão đáp xuống vùng biển của chúng ta.","Lũ lụt ở đồng bằng sông Cửu Long đến chủ yếu từ: |Mưa mạnh, thủy triều pho mát |Mưa tập trung vào một mùa giải |Đồng bằng cô gái thấp |Không có đê lũ lụt",,,,0.0
1,"Biểu hiện nào sau đây không đúng với khí hậu của Đồng bằng sông Cửu Long|Thiên tai bão, lũ quét, sạt lở đất diễn ra thường xuyên.|Lượng mưa lớn tập trung vào các tháng mùa mưa: tháng V – XI.|Chế độ nhiệt cao, ổn định quanh nắm.|Khí hậu cân xích đạo.","So với Đồng bằng sông Hồng, thiên nhiên Đồng bằng sông Cửu Long|được khai thác sớm hơn.|ít thay đổi hơn.|có một số vùng vẫn chưa bị tác động nhiều.|bị suy thoái nghiêm trọng.","Thiên tai bão, lũ quét, sạt lở đất diễn ra thường xuyên.",có một số vùng vẫn chưa bị tác động nhiều.,"0,8024098",0.0
2,"Cơ sở Địa lý Atlas Trang 9, cho biết tần suất của các hoạt động bão nhất vào khu vực?|.Delta sông Hồng.|.Trung tâm phía bắc.|.Hải Nam Trung Đông.|.Đồng bằng sông Cửu Long.","Dựa trên địa lý của Việt Nam Atlat, vui lòng cho biết khu vực có lượng mưa trung bình hàng năm ở nước ta?|.Ninh Thuận.|.Lai Châu.|.TP.Tp.ho Chi Minh.|.Nghệ An.",,,,0.0
3,"những biểu hiện của dân số nước ta đang ngày càng già đi : | nhóm tuổi 0 -14 và 15 – 59 giảm giá nhanh , trên 60 tuổi tăng khá nhanh .| nhóm tuổi 0 – 14 và 15 – 59 tăng nhanh , trên 60 tuổi tăng chậm .| nhóm tuổi 0 – 14 giảm giá , nhóm tuổi 15 – 59 và trên 60 tuổi tăng .| nhóm tuổi 0 -14 và trên 60 tăng lên , nhóm tuổi 15 – 59 giảm giá .",dựa vào bảng số liệu dưới đây : | dân cư nước ta ngày càng giảm .| dân cư nước ta tăng nhanh nhưng còn nhiều biến động | thời kì 1956 - 1960 có tỉ lệ tăng dân cư hằng năm cao nhất .| thời kì 1960 - 1985 có dân cư tăng trung bình hằng năm cao nhất .,"Nhóm tuổi 0 – 14 giảm, nhóm tuổi 15 – 59 và trên 60 tuổi tăng.",Thời kì 1960 - 1985 có dân số tăng trung bình hằng năm cao nhất.,,1.0
4,"Đất ở đồng bằng ven biển miền trung có đặc điểm kém, nhiều cát, ít phù sa hơn do: |Khi hình thành đồng bằng, biển đóng vai trò chính |Xói mòn, rửa trôi mạnh trong điều kiện mưa lớn | bởi những ngọn núi nằm dưới chân núi, nhận rất nhiều sỏi, cát nổi.|.Các dòng sông trung tâm ngắn, hẹp và rất nghèo.","Ở đồng bằng sông Cửu Long của mùa khô, nước thủy triều lấn chiếm gần hai phần ba diện tích độ mặn bị ô nhiễm, chủ yếu là do: |Có một mạng lưới kênh đào.|.Đất thấp, không có phong bì đê.|.Có nhiều khu vực rộng lớn của vùng thấp.|.Máy khắc serid 3 đồng bằng.",,,,0.0
...,...,...,...,...,...,...
16327,"đâu là đặc điểm đúng với hoạt động của bạo lực ở việt nam ?| mùa bạo lực bắt đầu từ tháng iv và kết thúc vào tháng xi .| mùa bạo lực chậm dần từ nam ra bắc .| 70% số cơn bạo lực trong mùa tập trung vào các tháng viii , ix , x .| trung bình mỗi năm có 10 đến 12 cơn bạo lực đổ bộ vào vùng biển nước ta .",nguyên nhân chủ yếu làm cho mùa bão nước ta chậm dần từ bắc vào nam là | hình dạng lãnh thổ hẹp ngang và kéo dài theo chiều bắc - nam .| gió mùa đông bắc suy dần khi di chuyển xuống phía nam .| dải hội tụ nhiệt đới lùi dần từ bắc vào nam và đạt được của bão .| nước ta tiếp giáp với biển đông rộng lớn .,"70% số cơn bão trong mùa tập trung vào các tháng VIII, IX, X.",dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão.,,1.0
16328,"Dựa trên Atlas địa lý Việt Nam, vui lòng cho biết gió mùa đông nào thổi vào nước ta theo hướng?|.Tây Bắc.|.Đông Bắc |.Tây nam.|.Đông Nam.",Các đặc điểm khí hậu của bờ biển phía Nam Trung Bộ khác so với khu vực phía Nam là loại khí hậu khẩn cấp.|.Mùa đông bị ảnh hưởng mạnh mẽ bởi gió thương mại.|.Khí hậu chia thành hai mùa: mùa mưa và mùa khô.|.Mưa lớn trên bộ sưu tập - là.,,,,0.0
16329,"Theo nguồn gốc của sự hình thành, địa hình ở khu vực đồng bằng của nước ta bao gồm các 

In [16]:
dataset = merged_df.set_axis(['sentence1', 'sentence2', 'ans_sentence1', 'ans_sentence2', 'similarity', 'label'], axis=1, inplace=False)

In [17]:
dataset

,sentence1,sentence2,ans_sentence1,ans_sentence2,similarity,label
0,"Đâu là đặc điểm thực sự của các hoạt động bão tại Việt Nam?|.Mùa bão bắt đầu từ tháng 1 và kết thúc vào tháng 11.|.Mùa bão chậm chậm từ phía nam đến phía bắc .70% những cơn bão trong mùa được tập trung vào những tháng VIII, IX, X. |Trung bình, 10 đến 12 cơn bão đáp xuống vùng biển của chúng ta.","Lũ lụt ở đồng bằng sông Cửu Long đến chủ yếu từ: |Mưa mạnh, thủy triều pho mát |Mưa tập trung vào một mùa giải |Đồng bằng cô gái thấp |Không có đê lũ lụt",,,,0.0
1,"Biểu hiện nào sau đây không đúng với khí hậu của Đồng bằng sông Cửu Long|Thiên tai bão, lũ quét, sạt lở đất diễn ra thường xuyên.|Lượng mưa lớn tập trung vào các tháng mùa mưa: tháng V – XI.|Chế độ nhiệt cao, ổn định quanh nắm.|Khí hậu cân xích đạo.","So với Đồng bằng sông Hồng, thiên nhiên Đồng bằng sông Cửu Long|được khai thác sớm hơn.|ít thay đổi hơn.|có một số vùng vẫn chưa bị tác động nhiều.|bị suy thoái nghiêm trọng.","Thiên tai bão, lũ quét, sạt lở đất diễn ra thường xuyên.",có một số vùng vẫn chưa bị tác động nhiều.,"0,8024098",0.0
2,"Cơ sở Địa lý Atlas Trang 9, cho biết tần suất của các hoạt động bão nhất vào khu vực?|.Delta sông Hồng.|.Trung tâm phía bắc.|.Hải Nam Trung Đông.|.Đồng bằng sông Cửu Long.","Dựa trên địa lý của Việt Nam Atlat, vui lòng cho biết khu vực có lượng mưa trung bình hàng năm ở nước ta?|.Ninh Thuận.|.Lai Châu.|.TP.Tp.ho Chi Minh.|.Nghệ An.",,,,0.0
3,"những biểu hiện của dân số nước ta đang ngày càng già đi : | nhóm tuổi 0 -14 và 15 – 59 giảm giá nhanh , trên 60 tuổi tăng khá nhanh .| nhóm tuổi 0 – 14 và 15 – 59 tăng nhanh , trên 60 tuổi tăng chậm .| nhóm tuổi 0 – 14 giảm giá , nhóm tuổi 15 – 59 và trên 60 tuổi tăng .| nhóm tuổi 0 -14 và trên 60 tăng lên , nhóm tuổi 15 – 59 giảm giá .",dựa vào bảng số liệu dưới đây : | dân cư nước ta ngày càng giảm .| dân cư nước ta tăng nhanh nhưng còn nhiều biến động | thời kì 1956 - 1960 có tỉ lệ tăng dân cư hằng năm cao nhất .| thời kì 1960 - 1985 có dân cư tăng trung bình hằng năm cao nhất .,"Nhóm tuổi 0 – 14 giảm, nhóm tuổi 15 – 59 và trên 60 tuổi tăng.",Thời kì 1960 - 1985 có dân số tăng trung bình hằng năm cao nhất.,,1.0
4,"Đất ở đồng bằng ven biển miền trung có đặc điểm kém, nhiều cát, ít phù sa hơn do: |Khi hình thành đồng bằng, biển đóng vai trò chính |Xói mòn, rửa trôi mạnh trong điều kiện mưa lớn | bởi những ngọn núi nằm dưới chân núi, nhận rất nhiều sỏi, cát nổi.|.Các dòng sông trung tâm ngắn, hẹp và rất nghèo.","Ở đồng bằng sông Cửu Long của mùa khô, nước thủy triều lấn chiếm gần hai phần ba diện tích độ mặn bị ô nhiễm, chủ yếu là do: |Có một mạng lưới kênh đào.|.Đất thấp, không có phong bì đê.|.Có nhiều khu vực rộng lớn của vùng thấp.|.Máy khắc serid 3 đồng bằng.",,,,0.0
...,...,...,...,...,...,...
16327,"đâu là đặc điểm đúng với hoạt động của bạo lực ở việt nam ?| mùa bạo lực bắt đầu từ tháng iv và kết thúc vào tháng xi .| mùa bạo lực chậm dần từ nam ra bắc .| 70% số cơn bạo lực trong mùa tập trung vào các tháng viii , ix , x .| trung bình mỗi năm có 10 đến 12 cơn bạo lực đổ bộ vào vùng biển nước ta .",nguyên nhân chủ yếu làm cho mùa bão nước ta chậm dần từ bắc vào nam là | hình dạng lãnh thổ hẹp ngang và kéo dài theo chiều bắc - nam .| gió mùa đông bắc suy dần khi di chuyển xuống phía nam .| dải hội tụ nhiệt đới lùi dần từ bắc vào nam và đạt được của bão .| nước ta tiếp giáp với biển đông rộng lớn .,"70% số cơn bão trong mùa tập trung vào các tháng VIII, IX, X.",dải hội tụ nhiệt đới lùi dần từ bắc vào nam và hoạt động của bão.,,1.0
16328,"Dựa trên Atlas địa lý Việt Nam, vui lòng cho biết gió mùa đông nào thổi vào nước ta theo hướng?|.Tây Bắc.|.Đông Bắc |.Tây nam.|.Đông Nam.",Các đặc điểm khí hậu của bờ biển phía Nam Trung Bộ khác so với khu vực phía Nam là loại khí hậu khẩn cấp.|.Mùa đông bị ảnh hưởng mạnh mẽ bởi gió thương mại.|.Khí hậu chia thành hai mùa: mùa mưa và mùa khô.|.Mưa lớn trên bộ sưu tập - là.,,,,0.0
16329,"Theo nguồn gốc của sự hình thành, địa hình ở khu vực đồng bằng của nước ta bao gồm các loại: |Đồng bằng d

**Train Test Split**

In [18]:
!pip install sklearn
from sklearn.model_selection import train_test_split

In [19]:
df_train, df_test = train_test_split(dataset, test_size=0.2, random_state = 8, shuffle=True)
df_train, df_val = train_test_split(dataset, test_size=0.25, random_state = 8)

In [20]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(12249, 6)
(4083, 6)
(3267, 6)


In [21]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [22]:
df_train

,sentence1,sentence2,ans_sentence1,ans_sentence2,similarity,label
0,"ở miền bắc , đường kẻ nhiệt đới gió mùa có độ cao trung bình dưới ( m ) : | 400 – 500 .| 500 – 600 .| 600 – 700 .| 700 – 800 .","ở miền nam nước ta , đai nhiệt đới gió lên đến độ cao ?| 600-700 m .| 700-800 m .| 800-900 m .| 900-1000 m .",600 – 700.,900-1000m.,,1.0
1,Những lĩnh vực nào sau đây nuôi dưỡng hầu hết nước ta?|.Trung tâm phía bắc.|.Midlands và núi phía Bắc.|.Bờ biển Nam Trung Bộ.|.Đồng bằng sông Cửu Long.,Đối với bảng dữ liệu: |Giá trị xuất khẩu của nước ta đang tăng nhanh.|.Giá trị xuất khẩu của các khu vực để đầu tư nội bộ và nước ngoài tăng.|.Giá trị xuất khẩu hàng hóa Vùng quốc gia đang gia tăng nhanh chóng rằng ngành đã đầu tư ra nước ngoài.|.Giá trị xuất khẩu của khu vực đầu tư ở nước ngoài có xu hướng ngày càng chiếm ưu thế hơn so với khu vực kinh tế quốc gia.,,,,0.0
2,"chất lượng nguồn lực lượng lao động nước ta ngày càng được nâng cao là nhờ | số lượng lực lượng lao động làm việc trong các công ti liên doanh tăng lên .| những thành tựu trong phát triển văn hoá , giáo dục , y tế .| mở thêm nhiều trung tâm đào tạo , hướng nghiệp .| phát triển công nghiệp , dịch vụ ở nông thôn .","hướng giải quyết việc làm nào dưới đây chủ yếu tập trung vào vấn đề con người ?| tăng cường xuất khẩu các mặt hàng nông sản .| nâng cao chất lượng đội ngũ người lao động .| đa dạng hoá các hoạt động sản xuất công - nông .| hợp tác với các nước phát ra , thu hút vốn đầu tư .","những thành tựu trong phát triển văn hóa, giáo dục, y tế.",Nâng cao chất lượng đội ngũ người lao động.,,1.0
3,"ở miền bắc , đai nhiệt đới gió có độ cao trung bình dưới ( m ) : | 400 – 500 .| 500 – 600 .| 600 – 700 .| 700 – 800 .","ở miền nam nước ta , đai nhiệt đới gió mùa lên đến mức độ cao ?| 600-700 m .| 700-800 m .| 800-900 m .| 900-1000 m .",600 – 700.,900-1000m.,,1.0
4,"Những khó khăn chính làm tăng chi phí xây dựng và bảo trì mạng lưới giao thông ở nước ta: |Khí hậu nhiệt đới giữ ẩm cho gió mùa, mưa lớn tập trung theo mùa.|.Nhiều địa hình với những ngọn đồi và thảm họa không thường xuyên, mưa lớn tập trung vào mùa.|.Thiếu vốn đầu tư, nền tảng kỹ thuật của ngành vẫn còn yếu.|.Đội ngũ kỹ sư và công nhân kỹ thuật của ngành đã không gặp các yêu cầu phát triển của ngành.",Đường sắt dài nhất ở nước ta là: |Hà Nội - Hải Phòng.|.Hà Nội - Lào Cai.|.Hà Nội - Tp.ho Chi Minh.|.Hà Nội - Thái Nguyên.,,,,0.0
...,...,...,...,...,...,...
12244,"lao động nước ta chủ yếu tập trung ở các ngành nông – lâm – thuỷ sản là do | các ngành này có cơ cấu đa dạng , trình độ sản xuất cao .| thực hiện đa dạng hoá các hoạt động sản xuất ở nông thôn .| sử dụng nhiều máy móc trong sản xuất .| tỉ lệ lao động thủ công còn cao , sử dụng công cụ thô sơ vẫn còn phổ biến .","căn cứ vào atlat địa lí việt nam trang 15 , hãy cho biết từ năm 1995 đến năm 2007 , sự chuyển dịch cơ cấu lao động đang làm việc theo khu vực kinh tế nào sau đây không đúng ?| tỉ trọng lao động nông , lâm , thuỷ sản giảm .| tỉ trọng lao động công nghiệp và xây dựng tăng .| tỉ trọng lao động dịch vụ tăng .| tỉ trọng lao động dịch vụ luôn nhỏ nhất .","tỉ lệ lao động thủ công còn cao, sử dụng công cụ thô sơ vẫn còn phổ biến.",Tỉ trọng lao động dịch vụ luôn nhỏ nhất.,,1.0
12245,"Tây Nguyên không phải là một khu vực: |Có cao nguyên Badan lớn, ở độ cao khác nhau.|.Nhiều vùng đất đỏ đá vôi đỏ và đất xám bạc trên Silt cổ.|.Khí hậu chia hai mùa mưa - khô rõ ràng.|.Thiếu nước cho mùa khô.",Đào tạo các khu vực chuyên ngành đã cho thấy: |Sự phân bố của cây trồng phù hợp hơn cho các khu vực sinh thái nông nghiệp.|.Sự phát triển của cấu trúc cây trồng phù hợp cho điều kiện sinh thái nông nghiệp.|.Các thác cáo có hiệu quả hơn nông nghiệp nhiệt đới của đất nước chúng ta.|.Cấu trúc của các nền văn hóa được đa dạng theo nhu cầu của thị trường.,,,,0.0
12246,"Khí hậu của đất nước chúng ta có một đặc tính khí hậu của Hải Dương, điều hòa không nhiều hơn: |Nằm gần xích đạo, mưa lớn.|.Địa hình 85% là những ngọn đồi núi thấp.|.Về tác động thường xuyên của gió mùa.| Với Biển Đông."

# Load Bert Model

In [23]:
from transformers import BertTokenizer, BertModel

In [24]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='bert-base-cased'):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = str(self.data.loc[index, 'sentence1'])
        sent2 = str(self.data.loc[index, 'sentence2'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

In [25]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="bert-base-cased", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        #  Instantiating BERT-based model object
        self.bert_layer = BertModel.from_pretrained(bert_model, return_dict=False)
        hidden_size = 768

        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        # Classification layer
        self.cls_layer = nn.Linear(hidden_size, 1)

        self.dropout = nn.Dropout(p=0.1)

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        # Feeding the inputs to the BERT-based model to obtain contextualized representations
        # cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)

        # Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
        # Linear Layer and a Tanh activation. The Linear layer weights were trained from the sentence order prediction (ALBERT) or next sentence prediction (BERT)
        # objective during pre-training.
        logits = self.cls_layer(self.dropout(pooler_output))
        
        return logits

In [26]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            print(f'logits = {logits}')
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            print(f'mean_loss = {mean_loss}')
            count += 1
            print(f'count = {count}')

    return (mean_loss/count)

In [27]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...
mkdir: cannot create directory ‘models’: File exists


In [28]:
bert_model = "bert-base-cased"
freeze_bert = False  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = 256
bs = 8
iters_to_accumulate = 2  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = 2e-5
epochs = 4

In [29]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []

    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if (val_loss < best_loss):
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    path_to_model='models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(bert_model, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

# Train and Evaluation

In [30]:
#  Set all seeds to make reproducible results
set_seed(1)

# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=5,shuffle=True)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=5,shuffle=True)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

Reading training data...
Reading validation data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 20%|██        | 307/1532 [00:52<02:55,  6.99it/s]


Iteration 306/1532 of epoch 1 complete. Loss : 0.11651216416917985 


 40%|████      | 613/1532 [01:37<02:14,  6.85it/s]


Iteration 612/1532 of epoch 1 complete. Loss : 0.053145972238602485 


...

logits = tensor([[ 9.2188],
        [ 9.2031],
        [-9.4141],
        [ 9.1641],
        [-9.4609],
        [-9.4453],
        [ 9.1797],
        [ 9.2031]], device='cuda:0', dtype=torch.float16)
mean_loss = 40.37291431427002
count = 505
logits = tensor([[-9.4531],
        [ 9.2422],
        [ 9.1797],
        [-1.3574],
        [ 9.1406],
        [ 9.2188],
        [-1.0693],
        [ 9.2344]], device='cuda:0', dtype=torch.float16)
mean_loss = 40.57217884063721
count = 506
logits = tensor([[ 0.3479],
        [-9.4922],
        [-1.1299],
        [ 8.3438],
        [-1.2988],
        [ 9.1797],
        [-9.4219],
        [-9.3125]], device='cuda:0', dtype=torch.float16)
mean_loss = 40.74765491485596
count = 507
logits = tensor([[-9.4609],
        [-9.4297],
        [-9.4531],
        [-9.4297],
        [ 9.2031],
        [ 9.1953],
        [-9.4219],
        [ 9.2500]], device='cuda:0', dtype=torch.float16)
mean_loss = 40.74765491485596
count = 508


100%|█████████▉| 509/511 [00:28<00:00, 19.06it/s]

logits = tensor([[ 9.2578],
        [ 9.1562],
        [ 9.1250],
        [ 9.1953],
        [-9.4141],
        [-1.1992],
        [-9.3828],
        [ 9.2422]], device='cuda:0', dtype=torch.float16)
mean_loss = 40.78061389923096
count = 509
logits = tensor([[-9.4141],
        [ 9.2656],
        [-9.5078],
        [-9.3750],
        [-9.4297],
        [ 9.0234],
        [ 9.2422],
        [ 9.2344]], device='cuda:0', dtype=torch.float16)
mean_loss = 40.78061389923096
count = 510
logits = tensor([[-0.8779],
        [ 9.2266],
        [-9.4297]], device='cuda:0', dtype=torch.float16)
mean_loss = 41.18922394514084
count = 511


100%|██████████| 511/511 [00:28<00:00, 17.91it/s]



Epoch 4 complete! Validation Loss : 0.08060513492199772
The model has been saved in models/bert-base-cased_lr_2e-05_val_loss_0.0797_ep_3.pt


# Predictions

In [32]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by applying the sigmoid function
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="results/output.txt"):
    """
    Predict the probabilities on a dataset with or without labels and print the result in a file
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, token_type_ids in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()

    w.writelines(str(prob)+'\n' for prob in probs_all)
    w.close()

In [33]:
path_to_model = '/content/models/bert-base-cased_lr_2e-05_val_loss_0.0797_ep_3.pt'  

path_to_output_file = 'results/output.txt'

print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)

model = SentencePairClassifier(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))

Reading test data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Loading the weights of the model...
Predicting on test data...


100%|██████████| 409/409 [00:20<00:00, 20.13it/s]


Predictions are available in : results/output.txt


# Evaluation

In [34]:
path_to_output_file = 'results/output.txt'  # path to the file with prediction probabilities

labels_test = df_test['label']  # true labels

probs_test = pd.read_csv(path_to_output_file, header=None)[0]  # prediction probabilities
threshold = 0.5   # you can adjust this threshold for your own dataset
preds_test=(probs_test>=threshold).astype('uint8') # predicted labels using the above fixed threshold

metric = load_metric("glue", "mrpc")

https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmpljehdhf3


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

storing https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py in cache at /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
creating metadata file for /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
Checking /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py for additional imports.
Creating main folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue
Creating specific version folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/mod

In [35]:
# Compute the accuracy and F1 scores
metric._compute(predictions=preds_test, references=labels_test)

{'accuracy': 0.9528619528619529, 'f1': 0.957622454595487}